In [6]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [7]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [8]:
root_path = "/content/drive/MyDrive/ml-data/DogCat202"

In [9]:
trans = transforms.Compose([
                            transforms.Resize((128, 128))
])
train_dataset = torchvision.datasets.ImageFolder(root=root_path, transform=trans)

In [10]:
type(train_dataset)

torchvision.datasets.folder.ImageFolder

In [14]:
for i, (data, label) in enumerate(train_dataset):
  print(i, data, label)
  if label == 0: # if cat image
    data.save(root_path+"/train_data/cat/%d_%d.jpeg"%(label, i))
  if label == 1:
    data.save(root_path+"/train_data/dog/%d_%d.jpeg"%(label, i))

0 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05B9D0> 0
1 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D0576D0> 0
2 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05ACD0> 0
3 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05A610> 0
4 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D0412D0> 0
5 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D08C790> 0
6 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05AF90> 0
7 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D041050> 0
8 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D057110> 0
9 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D041090> 0
10 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05B990> 0
11 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05A290> 0
12 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D05AF10> 0
13 <PIL.Image.Image image mode=RGB size=128x128 at 0x7F8D0D052FD0> 0
14 <PIL.Image.Image image mode=RGB size=128x

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(777)
if device == "cuda":
  torch.cuda.manual_seed_all(777)

In [16]:
trans = transforms.Compose([
                            transforms.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(root=root_path+"/train_data", transform=trans)

In [20]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

In [39]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.layer3 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(32768, 32),
        nn.ReLU(),
        nn.Linear(32,2)
    )
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    return out
model = CNN().to(device)

In [44]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss().to(device)

In [45]:
total_batch = len(train_dataloader)

epochs = 10
for epoch in range(epochs):
  avg_cost = .0
  for i, (data, target) in enumerate(train_dataloader):
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, target)
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch
  print(f"Epoch : {epoch+1} - Cost : {avg_cost}")

Epoch : 1 - Cost : 0.7148247361183167
Epoch : 2 - Cost : 0.6831355690956116
Epoch : 3 - Cost : 0.6479586362838745
Epoch : 4 - Cost : 0.6095252633094788
Epoch : 5 - Cost : 0.5497651100158691
Epoch : 6 - Cost : 0.4312179982662201
Epoch : 7 - Cost : 0.37094467878341675
Epoch : 8 - Cost : 0.28856855630874634
Epoch : 9 - Cost : 0.25179293751716614
Epoch : 10 - Cost : 0.17254161834716797
